# CSE 258 Fall 2017 - Homework 3
- - -

## Xiangyuan Ren
###  Department of Electrical and Computer Engineering 
### Email: xir010@eng.ucsd.edu 


---
## Tasks (Visit prediction)
### 1. Although we have built a validation set, it only consists of positive samples. For this task we also need examples of user/business pairs that weren’t visited. Build such a set by randomly sampling users and businesses until you have 100,000 non-visited user/business pairs. This random sample combined with your 100,000 validation reviews now corresponds to the complete validation set for the visit prediction task. Evaluate the performance (accuracy) of the baseline model on the validation set you have built (1 mark).
### 2.The existing ‘visit prediction’ baseline just returns True if the business in question is ‘popular,’ using a threshold of the 50th percentile of popularity (totalVisits/2). Assuming that the ‘non-visited’ test examples are a random sample of user-visit pairs, is this particular threshold value the best? If not, see if you can find a better one (and report its performance), or if so, explain why it is the best (1 mark).
* import data
* shuffle data
* building training data

In [2]:
import numpy as np
import math
from collections import defaultdict
def ParseDataFromFile(f):
    for l in open(f):
        yield eval(l)
data=list(ParseDataFromFile("assignment1_data/train.json"))
np.random.shuffle(data) 
train=data[:100000]

In [3]:
data[0]

{'businessID': 'B015691642',
 'categories': [u'Restaurant', u'Thai Restaurant', u'Filipino Restaurant'],
 'rating': 4.0,
 'reviewHash': 'R546396645',
 'reviewText': u'A venture by Top Chef alum Leah Cohen and there Fatty Crew, Pig and Khao serves up Filipino and Thai-influenced dishes in a cozy space in the Lower East Side. Grab a seat by the counter for a view of the kitchen at work, or pick from a few two-tops for a more intimate meal. The Sizzing Sisig is one of the more popular dishes, consisting of a variety of pork parts mixed with chilies and cilantro which are served in a hot skillet. A raw egg is mixed in and the remaining heat cooks it through. Worth a trip, especially for their brunch menu.',
 'reviewTime': u'Jan 17, 2014',
 'unixReviewTime': 1389989083,
 'userID': 'U094737036'}

In [2]:
#train=data[:100000]

* UserS,BuzS  :set{str} set that contain ID(str) of user and business
* UserS2N UserN2S: dict{str:int} dict{int: str} that translate string in number and translate number into string for user
* BuzS2N BuzN2S: dict{str:int} dict{int: str} that translate string in number and translate number into string for businessUserT 
* UserT BuzT: dict{str:int} that shows how many visits a user make and how many times a businiess been visited 

In [4]:

#21321 business - 20472 in train
#18793 users - 18085 in train
UserS   = set()
UserS2N = {}
UserN2S = {}
BuzS    = set()
BuzS2N  = {}
BuzN2S  = {}

for i in data:
    UserS.add(i['userID'])
    BuzS.add(i['businessID'])
    
j=0
for i in UserS:
    UserS2N[i]=j
    UserN2S[j]=i
    j+=1

j=0
for i in BuzS:
    BuzS2N[i]=j
    BuzN2S[j]=i
    j+=1

UserT   = defaultdict(int)
BuzT    = defaultdict(int)
for i in data:
    UserT[i['userID']]+=1
    BuzT[i['businessID']]+=1

* UserBlist,BuzUlist: dict{int:list} contains which buz a user visit, and which user visit the buz

In [5]:
UserBlist=defaultdict(list)
BuzUlist =defaultdict(list)
for i in data:
    u=UserS2N[i['userID']]
    b=BuzS2N[i['businessID']]
    UserBlist[u].append(b)
    BuzUlist[b].append(u)

* Making validation set
* UserBlist_V{(user int,buz int):1/0} : dict that shows whether user visit buz 1- visit, 0- not visit

In [6]:

import random
UserBlist_V={}
for i in data:
    u=UserS2N[i['userID']]
    b=BuzS2N[i['businessID']]
    UserBlist_V[(u,b)]=1

i=0
while i<200000:
    b=int(random.uniform(0,len(BuzS)))
    u=int(random.uniform(0,len(UserS)))
    if b not in UserBlist[u] and (u,b) not in UserBlist_V:
        UserBlist_V[(u,b)]=0
        i+=1

* testing baseline on  50th  percentile of popularity
* Popular : list (#popularity, buz) 
* totalPurchases= 100K

In [47]:
#testing baseline
Popular = [(BuzT[x],x) for x in BuzT]
Popular.sort()
Popular.reverse()

totalPurchases=100000
return1 = set()
count = 0
for ic, i in Popular:
    count += ic
    return1.add(BuzS2N[i])
    if count > totalPurchases*0.5: 
        break

miss=0
for i in UserBlist_V:
    predict=1 if (i[1] in return1)else 0
    miss+=(predict+UserBlist_V[i])%2
print "accuracy: ",1-miss*1.0/400000

accuracy:  0.5954075


### So, the baseline of threshold on 50% of total purchase get accuracy of 62.54%
---

In [119]:
for thres in [0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.8,1]:
    #Popular = [(BuzT[x],x) for x in BuzT]
    #Popular.sort()
    #Popular.reverse()

    #totalPurchases=100000
    return1 = set()
    count = 0
    for ic, i in Popular:
        count += ic
        return1.add(BuzS2N[i])
        if count > totalPurchases*thres: 
            break

    miss=0
    for i in UserBlist_V:
        predict=1 if (i[1] in return1)else 0
        miss+=(predict+UserBlist_V[i])%2
    print thres,'\t',
    print "accuracy: ",1-miss*1.0/400000

0.4 	accuracy:  0.579295
0.45 	accuracy:  0.587615
0.5 	accuracy:  0.5954075
0.55 	accuracy:  0.6027575
0.6 	accuracy:  0.60963
0.65 	accuracy:  0.6158875
0.7 	accuracy:  0.6220975
0.8 	accuracy:  0.633115
1 	accuracy:  0.6492925


### So, the 50% threshold is not the best
### with the  threshold on 60% of total purchase get better accuracy of 62.75%
---

### 3.Users may tend to repeatedly visit business of the same type. Build a baseline that returns ‘True’ if a user has visited a business of the same category before (at least one category in common), or zero otherwise (1 mark).
### 4. To run our model on the test set, we’ll have to use the files ‘pairs Visit.txt’ to find the userID/businessID pairs about which we have to make predictions. Using that data, run the above model and upload your solution to Kaggle. Tell us your Kaggle user name (1 mark). If you’ve already uploaded a better solution to Kaggle, that’s fine too!

In [49]:
# making a dict {buzID int: categories} 
BuzCate=defaultdict(set)
for i in data:
    b=BuzS2N[i['businessID']]
    for j in i['categories']:
        BuzCate[b].add(j)

In [50]:
# making a function to find categories in common using Jaccard similarity
def findcate(u,b):
    if u not in UserN2S:
        return 0
    if b not in BuzN2S: 
        return 0
    d=0
    for b2 in UserBlist[u]:
        d+=distance2(b,b2)
    return d

def distance2(b1,b2):
    if BuzT[BuzN2S[b2]]==0:   
        return 0
    set1=set(BuzCate[b1])
    set2=set(BuzCate[b2])
    return len(set1 & set2)*1.0/len(set1 | set2)

In [52]:
#test category
for thres in [0,0.1,0.2,0.5]:#,0.1,0.2]:
    miss=0
    for i in UserBlist_V:
        predict=1 if (findcate(i[0],i[1])>thres) else 0
        miss+=(predict+UserBlist_V[i])%2
    print thres,'\t',
    print "accuracy: ",1-(miss*1.0/400000)

0 	accuracy:  0.8492275
0.1 	accuracy:  0.8492275
0.2 	accuracy:  0.8705725
0.5 	accuracy:  0.93211


In [70]:
for thres in [1]:#,0.1,0.2]:
    miss=0
    for i in UserBlist_V:
        d=0
        for b2 in UserBlist[i[0]]:
            d+=distance1(b2,i[1])*1000
        predict=1 if (d>thres) else 0
        miss+=(predict+UserBlist_V[i])%2
    print thres,'\t',
    print "accuracy: ",1-(miss*1.0/400000)

1 	accuracy:  0.9512375


In [73]:
d0=[]
d1=[]
d2=[]
d3=[]
y =[]
for i in UserBlist_V:
    dd0=0;dd1=0;dd2=0
    for b2 in UserBlist[i[0]]:
        dd0+=distance1(b2,i[1])
        dd1+=distance2(b2,i[1])
    d0.append(dd0)#user距离
    d1.append(dd0*1.0/pu[i[0]])#user距离除以买的东西数量
    d2.append(p[i[1]])#item popular
    d3.append(dd1)#cate距离
    y.append(UserBlist_V[i])

In [74]:
len(y)

400000

In [143]:
for amp in [1,1.1,1.2,2,10,1000,1100,1500,10000000]:
    out=1-sum([((d0[i]*amp>1)+y[i])%2for i in range(400000)])*1.0/400000
    print amp,'\t',out

1 	0.99926
1.1 	0.99943
1.2 	0.9993725
2 	0.9986975
10 	0.98808
1000 	0.9512375
1100 	0.9512375
1500 	0.9512375
10000000 	0.9512375


In [141]:
for amp in [0.1,1,2,3,4,5,10,100,1000]:
    out=1-sum([((d1[i]*20*amp>1)+y[i])%2for i in range(400000)])*1.0/400000
    print amp,out

0.1 0.5042975
1 0.973405
2 0.9961025
3 0.993395
4 0.9899175
5 0.9867675
10 0.97586
100 0.953685
1000 0.951255


In [146]:
for amp in [0,1,1.5,2,2.5,3,4]:
    out=1-sum([((d3[i]*amp>1)+y[i])%2for i in range(400000)])*1.0/400000
    print amp,'\t',out

0 	0.5
1 	0.7820675
1.5 	0.9465875
2 	0.93211
2.5 	0.913175
3 	0.90845
4 	0.8915775
5 	0.8705725
6 	0.84927
7 	0.8492275
10 	0.8492275
100 	0.8492275


In [134]:
sum(d3)*1.0/len(d0)

1.4492725297619828

## ？

In [ ]:
i=(5152, 19073)
d=0
for b2 in UserBlist[i[0]]:
    d+=distance1(b2,i[1])
d

### When returning True if any categories in common found, the accuracy is 65.09%
---
* distance1 shows similarity in users
* using both similarity on dataset
* train model in different parameter

In [30]:
def distance1(b1,b2):#????
    set1=set(BuzUlist[b1])
    set2=set(BuzUlist[b2])
    try:
        return len(set1 & set2)*1.0/len(set1 | set2)
    except:
        return 0

In [31]:
d1=[[0]*len(BuzS) for i in range(len(BuzS))]
d2=[[0]*len(BuzS) for i in range(len(BuzS))]
for i in UserBlist_V:
    b=i[1]
    u=i[0]
    d=0
    for j in UserBlist[u]:
        d1[b][j]=distance1(b,j)
        d2[b][j]=distance2(b,j)
print "distance storage finished"

distance storage finished


In [27]:
UserT   = defaultdict(int)
BuzT    = defaultdict(int)
for i in data:
    BuzT[i['businessID']]+=1
    UserT[i['userID']]+=1
Popular = [(BuzT[x],x) for x in BuzT]    
p={}
for i in Popular:
    p[BuzS2N[i[1]]]=i[0]
pu={}
for i in UserT:
    pu[UserS2N[i]]=UserT[i]

In [28]:
len(p),len(BuzS)

(21321, 21321)

In [29]:
tttrain=[]
ytrain=[]
#theta=[0,1,0]
for theta[0] in [1]:
    for theta[1] in [1]:
        for theta[2] in [1]:
            miss=0
            for i in UserBlist_V:
                u=i[0];b=i[1]
                if u not in UserN2S:
                    miss+=(0+UserBlist_V[i])%2
                    continue
                if b not in BuzN2S:
                    miss+=(0+UserBlist_V[i])%2
                    continue
                duser=0
                dcate=0
                for b2 in UserBlist[u]:
                    duser+=d1[b][b2]*theta[0]
                    dcate+=d2[b][b2]*theta[1]
                tttrain.append([1,duser,dcate,p[b],pu[u],duser*1.0/pu[u]])
                ytrain.append(UserBlist_V[i])
#                 predict=1 if d>theta[2] else 0
#                 miss+=(predict+UserBlist_V[i])%2
#             print theta[0],theta[1],theta[2]
#             print "accuracy: ",1-(miss*1.0/200000)

In [33]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(tttrain, ytrain)

Y_pred = random_forest.predict(tttrain)

random_forest.score(tttrain, ytrain)

0.7522766666666667

In [42]:
int(random_forest.predict(tttrain[200000:200001]))

0

In [34]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

logreg.fit(tttrain, ytrain)

Y_pred = logreg.predict(tttrain)

logreg.score(tttrain, ytrain)

0.74817666666666671

In [30]:
#from sklearn import svm

In [31]:
# clf = svm.SVC(C=10)
# clf.fit(tttrain, ytrain)

In [32]:
# test_accuracy=sum([(x==y) for x,y in zip(tttrain,ytrain)])*1.0/len(ytrain)*100
# print test_accuracy

###  the accuracy is higher: 74%
---
### using it on the whole dataset

In [5]:
UserT   = defaultdict(int)
BuzT    = defaultdict(int)
for i in data:
    UserT[i['userID']]+=1
    BuzT[i['businessID']]+=1
UserBlist=defaultdict(list)
BuzUlist =defaultdict(list)
for i in data:
    u=UserS2N[i['userID']]
    b=BuzS2N[i['businessID']]
    UserBlist[u].append(b)
    BuzUlist[b].append(u)

In [8]:
def distance1(b1,b2):#????
    set1=set(BuzUlist[b1])
    set2=set(BuzUlist[b2])
    try:
        return len(set1 & set2)*1.0/len(set1 | set2)
    except:
        return 0
def distance2(b1,b2):
    if BuzT[BuzN2S[b2]]==0:   
        return 0
    set1=set(BuzCate[b1])
    set2=set(BuzCate[b2])
    return len(set1 & set2)*1.0/len(set1 | set2)

In [12]:
# UserBlist1={}
# for i in data:
#     u=UserS2N[i['userID']]
#     b=BuzS2N[i['businessID']]
#     UserBlist1[(u,b)]=1

In [35]:
# d1=[[0]*len(BuzS) for i in range(len(BuzS))]
# d2=[[0]*len(BuzS) for i in range(len(BuzS))]
# for u in UserBlist:
#     b=i[1]
#     u=i[0]
#     d=0
#     for j in UserBlist[u]:
#         d1[b][j]=distance1(b,j)
#         d2[b][j]=distance2(b,j)
# print "distance storage finished"

In [150]:
#theta=[1000,1.5,1]
predictions = open("predictions_Visiting.txt", 'w')
noUorB=0
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    predictions.write(u + '-' + b + ',')
    if u not in UserS2N or b not in BuzS2N:
        noUorB+=1
        predictions.write(str(0) + '\n')        
    else:
        u = UserS2N[u]
        b = BuzS2N[b]
        flag = 0
        duser=0
        dcate=0
        for j in UserBlist[u]:#j=b2
            duser+=distance1(b,j)#*theta[0]
            #dcate+=distance2(b,j)#*theta[1]
        #flag=int(random_forest.predict([[1,duser,dcate,p[b],pu[u],duser*1.0/pu[u]]]))
        flag = 1 if duser*150 >1 else 0 #theta[2]
        #flag = 1 if dcate*1.5 >1 else 0
        predictions.write(str(flag) + '\n')
predictions.close()
print noUorB

63


* 63 user or buz not shown in the dataset
## Now I am the best in the leaderboard (Nov. 3th), I achieved 87% accuracy
----

## Tasks (Rating prediction)
### 1.Let’s start by building our training/validation sets much as we did for the first task. This time building a validation set is more straightforward, you can simply use half of the data for validation, and do not need to randomly sample non-visited users/businesses.


In [6]:
import math
validation=data[100000:]

### 2.What is the performance of a trivial predictor
###  rating(user, item) = α
### on the validation set, and what is the value of α (1 mark)?

In [7]:
train[0]

{'businessID': 'B623658108',
 'categories': [u'Clothing Store', u"Women's Clothing Store", u'Shoe Store'],
 'rating': 3.0,
 'reviewHash': 'R581117063',
 'reviewText': u"This Zara is so big, but it's kind of overwhelming. It's also kind of dark in there, so make sure you watch you're going lest you run into something. The service is okay? They say hello but that's about the extent of their involvement with you until you check out. It goes without saying that the clothes are amazing though!",
 'reviewTime': u'Jun 22, 2013',
 'unixReviewTime': 1371936682,
 'userID': 'U635583657'}

In [16]:
alpha=sum([i['rating'] for i in train])/len(train)

In [17]:
MSE0=math.sqrt(sum([(i['rating']-alpha)**2 for i in validation])/len(validation))

In [18]:
MSE0

0.8625808567315142

### So, with only alpha included, alpha is 4.18588, RMSE is 0.8639757999500715
---

### 3.Fit a predictor of the form
### rating(user, item) ≃ alpha + beta_user + beta_item,
### by fitting the mean and the two bias terms as described in the lecture notes. Use a regularization parameter of λ = 1. Report the MSE on the validation set (1 mark).


In [19]:
UserRlist=defaultdict(list)# in training set User int: Rating float
BuzRlist=defaultdict(list)# in training set  User int: Rating float
for i in train:
    u=UserS2N[i['userID']]
    b=BuzS2N[i['businessID']]
    r=i['rating']
    UserRlist[u].append(r)
    BuzRlist[b].append(r)

In [20]:
UserRaverage=[0]*len(UserS)# list: average rating float
BuzRaverage=[0]*len(BuzS)

In [21]:
for i in UserRlist:
    if len(UserRlist[i])!=0:
        UserRaverage[i]=sum(UserRlist[i])*1.0/len(UserRlist[i])
for i in BuzRlist:
    if len(BuzRlist[i])!=0:
        BuzRaverage[i]=sum(BuzRlist[i])*1.0/len(BuzRlist[i])

In [22]:
ubr=defaultdict(list)# dict{ userID int: (userID buzID rating) } 
bur=defaultdict(list)# dict{  buzID int: (userID buzID rating) } 
for i in train:
    u=UserS2N[i['userID']]
    b=BuzS2N[i['businessID']]
    r=i['rating']
    ubr[u].append((u,b,r))
    bur[b].append((u,b,r))

In [15]:
alphaold=0
lam=1.0
while (abs(alpha-alphaold)>1e-8):
    alphaold=alpha
    alpha=sum([ sum(i[2]-UserRaverage[i[0]]-BuzRaverage[i[1]] for i in ubr[u]) for u in ubr])/len(train)
    for u in ubr:
        betau=sum(i[2]-alpha-BuzRaverage[i[1]] for i in ubr[u])/(lam+len(ubr[u]))
        UserRaverage[u]=betau
    for b in bur:
        betab=sum(i[2]-UserRaverage[i[0]]-alpha for i in bur[b])/(lam+len(bur[b]))
        BuzRaverage[b]=betab

KeyboardInterrupt: 

In [ ]:
MSE1=0
for i in validation:
    u=UserS2N[i['userID']]
    b=BuzS2N[i['businessID']]
    r=i['rating']
    MSE1+= (r - alpha - BuzRaverage[b] -UserRaverage[u])**2

In [ ]:
MSE1/=len(validation)
MSE1 = math.sqrt(MSE1)
MSE1

### lambda = 1.0 and RMSE: 0.8000199799312584
---
### Report the user and item IDs that have the largest and smallest values of beta (1 mark).


In [233]:
tmp1=[(UserRaverage[i],i) for i in range(len(UserRaverage)) if UserRaverage[i]!=0]

In [234]:
tmp2=[(BuzRaverage[i],i) for i in range(len(BuzRaverage)) if BuzRaverage[i]!=0]

In [236]:
a=[max(tmp1),min(tmp1),max(tmp2),min(tmp2)]

print a[0][0],UserN2S[a[0][1]]
print a[1][0],UserN2S[a[1][1]]
print a[2][0],BuzN2S[a[2][1]]
print a[3][0],BuzN2S[a[3][1]]

1.16174870406 U565835214
-2.31510836222 U378878648
1.08216024736 B093985406
-2.24398033386 B808255680


* MAX beta: User U565835214 : 1.16174870406
* MAX beta: Buz  B093985406 : 1.08216024736
* MIN  beta: User U378878648 : -2.31510836222
* MIN  beta: Buz  B808255680 : -2.24398033386

### Find a better value of lambda using your validation set. Report the value you chose, its MSE, and upload your solution to Kaggle by running it on the test data (1 mark).

In [237]:
for lam in [2,2.5,4,5,6]:
    alphaold=0
    alpha=sum([i['rating'] for i in train])/len(train)
    
    for i in UserRlist:
        if len(UserRlist[i])!=0:
            UserRaverage[i]=sum(UserRlist[i])*1.0/len(UserRlist[i])
    for i in BuzRlist:
        if len(BuzRlist[i])!=0:
            BuzRaverage[i]=sum(BuzRlist[i])*1.0/len(BuzRlist[i])
    j=0
    while (abs(alpha-alphaold)>1e-6):
        alphaold=alpha
        alpha=sum( [sum([ (i[2]-UserRaverage[i[0]]-BuzRaverage[i[1]]) for i in ubr[u]]) for u in ubr] )*1.0/len(train)
        for u in ubr:
            betau=sum(i[2]-alpha-BuzRaverage[i[1]] for i in ubr[u])/(lam+len(ubr[u]))
            UserRaverage[u]=betau
        for b in bur:
            betab=sum(i[2]-UserRaverage[i[0]]-alpha for i in bur[b])/(lam+len(bur[b]))
            BuzRaverage[b]=betab
        j+=1
    
    MSE1=0
    for i in validation:
        u=UserS2N[i['userID']]
        b=BuzS2N[i['businessID']]
        r=i['rating']
        MSE1+= (r - alpha - BuzRaverage[b] -UserRaverage[u])**2
    MSE1/=len(validation)
    MSE1=math.sqrt(MSE1)
    print "lam: ",lam,'\t',j,'\t',"alpha: ",alpha,"\tMSE: ",MSE1

# lam:  0 	2 	alpha:  -4.18111 	MSE:  1.33058794417
# lam:  0.1 	766 	alpha:  4.23267676712 	MSE:  0.879022928257
# lam:  1 	110 	alpha:  4.22188678821 	MSE:  0.800019974418
# lam:  1.5 	81 	alpha:  4.22109435537 	MSE:  0.788595740468
# lam:  3 	50 	alpha:  4.21981608503 	MSE:  0.777730937422
# lam:  6 	33 	alpha:  4.21793576219 	MSE:  0.778741347218
# lam:  9 	27 	alpha:  4.21621132091 	MSE:  0.78409147555
# lam:  10 	25 	alpha:  4.21566482307 	MSE:  0.785934296225
# lam:  12 	23 	alpha:  4.21461746606 	MSE:  0.789480189525
# lam:  100 	11 	alpha:  4.19625888896 	MSE:  0.837800031307
# lam:  1000.0 	7 	alpha:  4.18372121862 	MSE:  0.860034493429


lam:  2 	66 	alpha:  4.22058188091 	MSE:  0.782648063632
lam:  2.5 	57 	alpha:  4.22017547666 	MSE:  0.7794265575
lam:  4 	42 	alpha:  4.21915985952 	MSE:  0.776761359093
lam:  5 	37 	alpha:  4.21853891828 	MSE:  0.777392046172
lam:  6 	33 	alpha:  4.21793576219 	MSE:  0.778741347218


### lambda: 4 RMSE: 0.776761359093
---
### Below is running on the whole dataset and upload
### Now I rank the 2nd in the leaderboard.(RMSE=0.74579)

In [7]:
UserRlist=defaultdict(list)
BuzRlist=defaultdict(list)
ubr=defaultdict(list)
bur=defaultdict(list)
for i in data:
    u=UserS2N[i['userID']]
    b=BuzS2N[i['businessID']]
    r=i['rating']
    UserRlist[u].append(r)
    BuzRlist[b].append(r)
    ubr[u].append((u,b,r))
    bur[b].append((u,b,r)) 

UserRaverage=[0]*len(UserS)
BuzRaverage=[0]*len(BuzS)

for i in UserRlist:
    if len(UserRlist[i])!=0:
        UserRaverage[i]=sum(UserRlist[i])*1.0/len(UserRlist[i])
for i in BuzRlist:
    if len(BuzRlist[i])!=0:
        BuzRaverage[i]=sum(BuzRlist[i])*1.0/len(BuzRlist[i]) 
          
    
    
lam=4
alphaold=0
alpha=sum([i['rating'] for i in data])/len(data)


j=0
while (abs(alpha-alphaold)>1e-6):
    alphaold=alpha
    alpha=sum( [sum([ (i[2]-UserRaverage[i[0]]-BuzRaverage[i[1]]) for i in ubr[u]]) for u in ubr] )*1.0/len(data)
    for u in ubr:
        betau=sum(i[2]-alpha-BuzRaverage[i[1]] for i in ubr[u])/(lam+len(ubr[u]))
        UserRaverage[u]=betau
    for b in bur:
        betab=sum(i[2]-UserRaverage[i[0]]-alpha for i in bur[b])/(lam+len(bur[b]))
        BuzRaverage[b]=betab
    j+=1



In [8]:
MSE1=0
for i in data:
    u=UserS2N[i['userID']]
    b=BuzS2N[i['businessID']]
    r=i['rating']
    MSE1+= (r - alpha - BuzRaverage[b] -UserRaverage[u])**2
MSE1/=len(data)
MSE1=math.sqrt(MSE1)
print "lam: ",lam,'\t',j,'\t',"alpha: ",alpha,"\tMSE: ",MSE1

NameError: name 'math' is not defined

In [27]:
#noUnoB=0
noUyesB=0
yesUnoB=0
predictions = open("predictions_Rating.txt", 'w')

for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    predictions.write(u + '-' + b + ',')
#     if u not in UserS2N and b not in BuzS2N: # 0
#         noUandB+=1
    if u not in UserS and b in BuzS: #11
        #tmp=sum(BuzRlist[BuzS2N[b]])*1.0/len(BuzRlist[BuzS2N[b]])
        b = BuzS2N[b]
        tmp=alpha+BuzRaverage[b]
        if tmp>5:
            tmp=5
        predictions.write(str(tmp) + '\n')
        print '0',tmp
        noUyesB+=1
        continue
    if u in UserS and b not in BuzS:#13
        #tmp=sum(UserRlist[UserS2N[u]])*1.0/len(UserRlist[UserS2N[u]])
        u = UserS2N[u]
        tmp=alpha+UserRaverage[u]
        if tmp>5:
            tmp=5
        predictions.write(str(tmp) + '\n')  
        print '1',tmp
        yesUnoB+=1  
        continue
    
    if u in 
    u = UserS2N[u]
    b = BuzS2N[b]
    tmp=alpha+UserRaverage[u]+BuzRaverage[b]
    if tmp>5:
        tmp=5
    predictions.write(str(tmp) + '\n')
predictions.close()
print noUyesB,yesUnoB

1 3.62697160518
1 4.03788213092
1 4.29469870205
0 3.82819169286
1 3.77966186143
1 4.07635256601
0 3.62554971221
1 4.41190180667
0 4.3202928927
1 3.87102297365
1 4.32825558809
1 4.35376940127
0 4.4426834473
0 4.21417803686
0 3.97626566821
0 4.28324095531
0 4.02763027263
1 3.81634513698
1 4.4309821432
0 4.48187352949
1 4.55299371942
0 4.50766998148
1 3.77534694104
0 4.36683285414
11 13
